In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch

es= Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tdRTedevS6WZDfd1oEOoeQ',
 'name': 'd3f17afee3cf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-10T10:08:26.972230187Z',
             'build_flavor': 'default',
             'build_hash': 'd4b391d925c31d262eb767b8b2db8f398103f909',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.1'}}


In [2]:
es.indices.delete(index='products', ignore_unavailable=True)
es.indices.create(index='products')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'})

In [3]:
import json 
from tqdm import tqdm 

document_ids = [] 
dummy_data = json.load(open("data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='products', body=document)
    document_ids.append(response['_id'])

100%|██████████| 3/3 [00:00<00:00, 41.60it/s]


In [4]:
document_ids

['PQV29JQBIRImumIu3TPQ', 'PgV29JQBIRImumIu3jMD', 'PwV29JQBIRImumIu3jMO']

In [5]:
# update existing 

from pprint import pprint 

response = es.update(
    index='products',
    id=document_ids[0],
    script={
        "source": "ctx._source.title = params.title",
        "params":{
            "title": "New Title"
        }
    },
)

pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


In [6]:
response = es.get(index='products', id=document_ids[0])
pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 3,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 2,
 'found': True}


In [7]:
# add new field using Script 

response = es.update(
    index='products',
    id=document_ids[0],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)

pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 3,
 'result': 'updated'}


In [8]:
response = es.get(index='products', id=document_ids[0])
pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 4,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'dummy_value',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 3,
 'found': True}


In [9]:
# add new field using Doc

response = es.update(
    index='products',
    id=document_ids[0],
    doc={
        "new_value_2": "dummy_value_2",
    },
)

pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 4,
 'result': 'updated'}


In [10]:
response = es.get(index='products', id=document_ids[0])
pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 5,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'dummy_value',
             'new_value_2': 'dummy_value_2',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 4,
 'found': True}


In [11]:
# remove field 

response = es.update(
    index='products',
    id=document_ids[0],
    script={
        "source": "ctx._source.remove('new_field')",
    },
)

pprint(response.body)


{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 6,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 5,
 'result': 'updated'}


In [12]:

response = es.get(index='products', id=document_ids[0])
pprint(response.body)

{'_id': 'PQV29JQBIRImumIu3TPQ',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 6,
 '_source': {'created_on': '2024-09-22',
             'new_value_2': 'dummy_value_2',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 5,
 'found': True}


In [13]:
# if document doesnt exists in the index - it is created 

response = es.update(
    index='products',
    id='1',
    doc={
        "book_id": 123456789,
        "book_title": "A book",
    },
    doc_as_upsert=True,
)

In [14]:
pprint(response.body)

{'_id': '1',
 '_index': 'products',
 '_primary_term': 1,
 '_seq_no': 7,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [15]:
response = es.count(index='products')
response['count']

4